This block imports all necessary libraries for the notebook.

pandas: For data manipulation and analysis.
baseflow: For streamflow separation.
matplotlib and seaborn: For data visualization.
csv: For reading and writing CSV files.
os: For operating system dependent functionality, like creating directories.
numpy: For numerical operations.
dataretrieval.nwis: For retrieving data from the USGS NWIS database.
math: For mathematical operations.

In [ ]:
import pandas as pd
import baseflow
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import os
import numpy as np
import dataretrieval.nwis as nwis
import math

This function 'process_baseflow_data' processes streamflow data to separate baseflow from it.

Input: Accepts a filepath to a CSV file containing streamflow data and an optional drainage area (default is 276).

Operation:

The function reads the CSV file into a DataFrame.
The streamflow data and dates are extracted from the DataFrame.
The baseflow separation is performed using the baseflow library.
The separated baseflow data is saved to an output CSV file named separated_baseflow.csv.
The Kling-Gupta Efficiency scores (KGEs) for the separation methods are calculated and saved to an output CSV file named KGEs.csv.
Output: The function returns the name of the best baseflow separation method based on the highest KGE score.

In [ ]:

def process_baseflow_data(filepath, area=276):
    # Read the CSV file directly into a DataFrame
    df = pd.read_csv(filepath, parse_dates=[0])

    # Split the DataFrame into 'Q', 'date'
    Q = df.iloc[:, 1]
    date = df.iloc[:, 0]

    # Perform baseflow separation
    b, KGEs = baseflow.separation(Q, date, area=area)
    
    # Save the separated baseflow data with dates
    baseflow_data = pd.DataFrame(b, index=date)
    baseflow_data.to_csv("separated_baseflow.csv")

    # Save Kling-Gupta Efficiency scores 'KGEs' with method names
    method_names = b.dtype.names
    KGEs_data = pd.DataFrame({'Method': method_names, 'KGEs': KGEs})
    KGEs_data.to_csv("KGEs.csv", index=False)
    
    # Return the best method
    best_method = method_names[KGEs.argmax()]
    return best_method


Function Explanation: separate_baseflow_multiple_columns

This function processes a CSV file containing streamflow data from multiple columns and separates baseflow for each.

Input: Takes a filepath to the CSV containing streamflow data and an optional drainage area parameter (default is 276).

Operation:

The CSV file is read into a DataFrame.
Streamflow data (Q1 and Q2) and dates are extracted for two columns.
Baseflow separation is executed for each streamflow column.
The separated baseflows and their respective Kling-Gupta Efficiency scores (KGEs) are prepared for saving.
The separated baseflows and KGEs are saved into two separate Excel files, with each sheet representing a column from the original CSV.
Output: Returns a message indicating successful saving of the data into the specified Excel files.

In [ ]:
import baseflow
import pandas as pd

def separate_baseflow_multiple_columns(filepath, area=276):
    # Read the CSV file directly into a DataFrame
    df = pd.read_csv(filepath, parse_dates=[0])

    # Split the DataFrame into 'Q', 'date' for each column
    Q1 = df.iloc[:, 1]
    Q2 = df.iloc[:, 2]
    date = df.iloc[:, 0]

    # Perform baseflow separation for each column
    b1, KGEs1 = baseflow.separation(Q1, date, area=area)
    b2, KGEs2 = baseflow.separation(Q2, date, area=area)

    # Prepare data for saving
    baseflow_data1 = pd.DataFrame(b1, index=date)
    baseflow_data2 = pd.DataFrame(b2, index=date)
    KGEs_data1 = pd.DataFrame({'Method': b1.dtype.names, 'KGEs': KGEs1})
    KGEs_data2 = pd.DataFrame({'Method': b2.dtype.names, 'KGEs': KGEs2})

    # Get the column headers for sheet names
    sheet_name1, sheet_name2 = df.columns[1], df.columns[2]

    # Save data to Excel files with different sheets
    with pd.ExcelWriter("separated_baseflowTest2.xlsx") as writer:
        baseflow_data1.to_excel(writer, sheet_name=sheet_name1)
        baseflow_data2.to_excel(writer, sheet_name=sheet_name2)

    with pd.ExcelWriter("KGEsTest2.xlsx") as writer:
        KGEs_data1.to_excel(writer, sheet_name=sheet_name1)
        KGEs_data2.to_excel(writer, sheet_name=sheet_name2)
        
    return "Data saved to separated_baseflowTest2.xlsx and KGEsTest2.xlsx"


Function Explanation: plot_baseflow_data

This function visualizes streamflow data alongside separated baseflow data for two columns, allowing the user to specify a separation method for comparison.

Input:

Q1, Q2: Streamflow data from two columns.
baseflow_data1, baseflow_data2: Baseflow separated data corresponding to Q1 and Q2.
date: Date values to be used on the x-axis.
method: The baseflow separation method to plot.
Operation:

Aesthetic settings for the plot are established using seaborn.
Two subplots are initialized for visualizing the streamflow and baseflow data side by side.
For each column, the streamflow data and the separated baseflow (using the specified method) are plotted against the provided dates.
Output:

Generates a side-by-side plot showing streamflow and the specified method's separated baseflow for both columns.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import baseflow
import pandas as pd

def plot_baseflow_data(Q1, baseflow_data1, Q2, baseflow_data2, date, method):
    # Set style for better aesthetics
    sns.set(style="white")

    # Create subplots with 2 columns
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))

    # Data for plotting
    data = [(Q1, baseflow_data1, "Column 1", axs[0]), (Q2, baseflow_data2, "Column 2", axs[1])]

    for Q, baseflow_data, title, ax in data:
        # Convert Series or lists to numpy arrays to avoid problematic indexing
        Q = np.array(Q)
        baseflow_selected_method = np.array(baseflow_data[method])
        date_array = np.array(date)

        ax.plot(date_array, Q, label='Discharge', linewidth=2)
        ax.plot(date_array, baseflow_selected_method, label=f'{method} Baseflow', linestyle='--', linewidth=2)
        ax.set_title(title, fontsize=14)
        ax.tick_params(axis='both', which='both', direction='in')
        ax.legend()
        ax.set_ylabel('Flow ($m^3/s$)')
        ax.set_xlabel('Date')

    plt.style.use('default')
    plt.show()


Function Explanation: retrieve_and_save_gage_data

This function fetches gage data from the nwis service, processes it, and saves the results into specified CSV files.

Input:

gage_numbers: A list of gage numbers for which data should be fetched.
output_file: Name of the CSV file to save the processed gage data.
nan_gages_file: Name of the CSV file to save gages with no data (NaN values).
valid_gages_file: Name of the CSV file to save valid gage numbers.
Operation:

Initialize empty lists to track gages with missing data and valid gages.
For each gage number in the provided list, retrieve its site information from the nwis service.
Process the retrieved data: extract drainage area, handle missing values, and convert units.
Save the processed data into the designated CSV files.
Output:

Returns a message indicating the success of the operation and the paths to the saved CSV files.

In [ ]:
import csv
import dataretrieval.nwis as nwis
import math

def retrieve_and_save_gage_data(gage_numbers, output_file, nan_gages_file, valid_gages_file):
    gages_with_nan = []
    valid_gages = []

    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Gage", "Area_sqmile", "Area_sqkm"])

        for gage_number in gage_numbers:
            formatted_gage = f"{gage_number}, USGS"
            
            try:
                site_info = nwis.get_record(sites=gage_number, service='site')
                drainage_area_sqmile = float(site_info.iloc[0]['drain_area_va'])

                if math.isnan(drainage_area_sqmile):
                    gages_with_nan.append(formatted_gage)
                    continue

                drainage_area_sqkm = drainage_area_sqmile * 2.58999
                writer.writerow([formatted_gage, drainage_area_sqmile, drainage_area_sqkm])
                valid_gages.append(formatted_gage)
            except Exception as e:
                print(f"Failed to retrieve data for site {gage_number}: {e}")

    with open(nan_gages_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Gage"])
        for gage in gages_with_nan:
            writer.writerow([gage])

    with open(valid_gages_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Gage"])
        for gage in valid_gages:
            writer.writerow([gage])

    return f"Data written to {output_file}. Gages with NaN values written to {nan_gages_file}. Valid gages written to {valid_gages_file}."


Function Explanation: convert_discharge_data

This function takes a CSV file with discharge data, converts the values using a specified conversion factor, and then writes the converted values to another CSV file.

Input:

input_file: The path to the input CSV file containing the original discharge data.
output_file: The path where the converted data should be saved.
valid_gages_file: A file containing a list of valid gage numbers for which data should be retained.
conversion_factor: The factor to use for converting the discharge values (default is 0.0283168).
Operation:

Reads the list of valid gage numbers from valid_gages_file.
Loads the data from the input_file into memory.
Filters out columns not listed in valid_gages_file.
Converts the discharge values using the provided conversion_factor.
Writes the converted data to the output_file.
Output:

Returns a message indicating the success of the operation and the path to the saved output_file.

In [ ]:
import csv
import pandas as pd

def convert_discharge_data(input_file, output_file, valid_gages_file, conversion_factor=0.0283168):
    header = []
    data = []

    with open(valid_gages_file, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # skip header
        valid_gages = [row[0] for row in reader]

    with open(input_file, mode='r') as file:
        reader = csv.reader(file)
        for row in reader:
            data.append(row)

    header = data[0]
    data = data[1:]

    col_indices_to_keep = [0] + [index for index, col in enumerate(header) if col in valid_gages]
    header = [header[i] for i in col_indices_to_keep]
    filtered_data = [[row[i] for i in col_indices_to_keep] for row in data]

    converted_data = []
    for row in filtered_data:
        new_row = [row[0]]
        for value in row[1:]:
            if value:
                new_row.append(str(float(value) * conversion_factor))
            else:
                new_row.append('')
        converted_data.append(new_row)

    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        for row in converted_data:
            writer.writerow(row)

    return f"Data converted and written to {output_file}."


Function Explanation: baseflow_separation_LA

This function performs baseflow separation on discharge data for multiple gages and saves the separated baseflow and Kling-Gupta Efficiency (KGE) scores to Excel files.

Input:

input_file: The path to the CSV file containing discharge data.
area_file: A file containing drainage areas for the gages.
baseflow_output: The path to save the separated baseflow data in an Excel format.
KGEs_output: The path to save the KGEs scores in an Excel format.
Operation:

Reads the discharge data from input_file and the drainage areas from area_file.
Iterates over each discharge column in the data.
For each gage, retrieves the drainage area and performs baseflow separation.
Saves the separated baseflow data and KGEs to Excel files with gage names as sheet names.
Output:

Returns a message confirming that the data has been saved to baseflow_output and KGEs_output.

In [ ]:
import pandas as pd
import baseflow

def baseflow_separation_LA(input_file, area_file, baseflow_output, KGEs_output):
    df = pd.read_csv(input_file, parse_dates=[0])
    date = df.iloc[:, 0]
    area_df = pd.read_csv(area_file)

    with pd.ExcelWriter(baseflow_output) as baseflow_writer, pd.ExcelWriter(KGEs_output) as KGEs_writer:
        for col in df.columns[1:]:
            gage_number = col.split(",")[0].strip()
            area = area_df.loc[area_df['Gage'].str.startswith(gage_number), 'Area_sqkm'].values[0]
            Q = df[col]
            b, KGEs = baseflow.separation(Q, date, area=area)
            baseflow_data = pd.DataFrame(b, index=date)
            KGEs_data = pd.DataFrame({'Method': b.dtype.names, 'KGEs': KGEs})
            baseflow_data.to_excel(baseflow_writer, sheet_name=col)
            KGEs_data.to_excel(KGEs_writer, sheet_name=col)
    
    return f"Data saved to {baseflow_output} and {KGEs_output}."


Function Explanation: convert_units_and_save

This function reads data from given Excel and CSV files, converts numerical values using a specified conversion factor, and then saves the converted data back to new Excel and CSV files.

Input:

excel_input: Path to the input Excel file containing data to be converted.
excel_output: Path where the converted Excel data will be saved.
csv_input: Path to the input CSV file containing data to be converted.
csv_output: Path where the converted CSV data will be saved.
conversion_factor: The factor used for converting the numerical values (default is 35.315).
Operation:

Opens the input Excel file and iterates over its sheet names.
For each sheet, parses the data and multiplies numerical values by the conversion factor.
Saves the converted data to the output Excel file.
Reads the input CSV file, applies the conversion to numerical columns, and then saves the converted data to the output CSV file.
Output:

Returns a message confirming that the data has been converted and saved to the specified output files.

In [ ]:
import pandas as pd

def convert_units_and_save(excel_input, excel_output, csv_input, csv_output, conversion_factor=35.315):
    xls_file = pd.ExcelFile(excel_input)
    with pd.ExcelWriter(excel_output) as writer:
        for sheet_name in xls_file.sheet_names:
            df = xls_file.parse(sheet_name)
            for col in df.columns:
                if df[col].dtype == 'float64':
                    df[col] = df[col] * conversion_factor
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    csv_file = pd.read_csv(csv_input)
    for col in csv_file.columns:
        if csv_file[col].dtype == 'float64':
            csv_file[col] = csv_file[col] * conversion_factor
    csv_file.to_csv(csv_output, index=False)

    return "Conversion completed and files saved."


Function Explanation: generate_and_save_plots

This function produces visual plots comparing the discharge and baseflow data from provided Excel and CSV files. It then saves these plots as image files in a designated directory.

Input:

baseflow_excel: Path to the input Excel file containing baseflow data for different sites.
discharge_csv: Path to the input CSV file containing discharge data for the same sites.
method: The baseflow separation method used, with "Local" as the default.
Operation:

Reads baseflow data from the given Excel file and discharge data from the given CSV file.
Iterates through each site (or column of data) from the CSV file.
For each site, it creates a plot comparing discharge and baseflow data.
The baseflow data used in the plots is derived from the provided method.
Saves each plot as an image in a directory named based on the chosen method.
Output:

Returns a message confirming that all plots have been generated and saved in the designated directory.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np

def generate_and_save_plots(baseflow_excel, discharge_csv, method="Local"):
    baseflow_file = pd.ExcelFile(baseflow_excel)
    discharge_df = pd.read_csv(discharge_csv)

    dates = np.array(pd.to_datetime(discharge_df.iloc[:, 0]))

    # Adjusted the save directory name to include the method
    save_dir = f"Discharge_and_Basflow_Plots_LA_{method}"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for idx, col in enumerate(discharge_df.columns[1:]):
        fig, ax = plt.subplots(figsize=(10, 4))
        discharge_series = np.array(discharge_df[col].values)
        baseflow_df = baseflow_file.parse(col)
        baseflow_series = np.array(baseflow_df[method].values)
        
        ax.plot(dates, discharge_series, label='Discharge', color='blue')
        ax.plot(dates, baseflow_series, label=f'Baseflow ({method})', color='red')

        ax.set_title(f"{col} - Discharge & Baseflow")
        ax.set_xlabel('Date')
        ax.set_ylabel('Flow ($ft^3$/s)')
        ax.legend()
        
        plt.show()
        
        save_path = os.path.join(save_dir, f"{col}_Discharge_Baseflow.png")
        fig.savefig(save_path)
        plt.close(fig)

    return f"All plots saved in '{save_dir}' directory."
